# COMP 2211 Exploring Artificial Intelligence
## Lab 9 Minimax and Alpha-Beta Pruning

## Lab tasks & Submission scheme

**Lab tasks:**
In this lab, you are required to implement a game, Math Tic-Tac-Toe, using Minimax and Alpha-Beta Pruning.

 **Submission to ZINC:**

 After completing all [TODO] in this notebook, successfully running the code, and achieving the task requirement, please **COPY** all your [TODO] to the template __lab9_task.py__ (do not change the file name). Then, zip __lab9_task.py__ and submit **ONLY** the .zip file to ZINC.

 Before you submit your codes, please check the ***indentations***.

 Arrangement for the ZINC test:
- test 1: correctness check: next move of length-4 moves on minimax

- test 2: correctness check: next move of length-3 moves on minimax

- test 3: correctness check: next move of length-2 moves on minimax

- test 4: correctness check: next move of length-2 moves on alpha-beta pruning

- test 5: running time check: alpha-beta pruning vs. minimax 




## Introduction

Math Tic-Tac-Toe is an innovative variant of the classic Tic-Tac-Toe game that integrates mathematical strategy into its gameplay. In this enhanced version, players use numbers from 1 to 9, instead of the traditional Xs and Os. The objective is to strategically place these numbers on a 3x3 grid to have the sum of any row, column, or diagonal equal exactly 15 using three numbers.

### Gameplay Example

Consider the following scenario in a game of Math Tic-Tac-Toe:

1. **Player 1** places the number **4** on the left of the top row.
2. **Player 2** responds by placing the number **9** in the middle of the grid.
3. **Player 1** then places the number **5** in the leftmost cell of the bottom row.
4. **Player 2** places the number **6** in the rightmost cell of the top row.

The board now looks like this:

```
| 4 |   | 6 |
|   | 9 |   |
| 5 |   |   |
```

5. **Player 1** can win by placing the number **2** in the rightmost cell of the bottom row, completing the diagonal from the bottom right to the top left (2 + 9 + 4 = 15).
```
| 4 |   | 6 |
|   | 9 |   |
| 5 |   | 2 |
```

This strategic placement of numbers, with the dual goals of reaching the sum of 15 and blocking the opponent, introduces a layer of depth and complexity unseen in the original game, making Math Tic-Tac-Toe a challenging game for players.

### Strategic Considerations

Players must think ahead, considering both offensive and defensive manoeuvres with each number placement. Strategic planning involves not just playing to win by completing a line but also ensuring that the opponent does not achieve the sum of 15 on their next turn.

### Special case for the first move

There is a special case when first move at (1,1) with 5. In this case, the first player will obviously win. Therefore, we remove this move from the available moves at the beginning of the game.


## Basic classes and functions
To avoid potential debugging issues in the future, please do not change the MathTicTacToe and Runner classes, as well as human_move and ai_move.


### Game Class

MathTicTacToe: a class to record game status and provide several useful functions, such as print_board, get_available_moves, etc.

print_run_time: a function wrapper that prints the running time of the function wrapped.

In [38]:
import numpy as np
import time
import random


def print_run_time(func):
    def wrapper(*args, **kw):
        local_time = time.time()
        output = func(*args, **kw)
        print('Current Function [%s] run time is %.2f s' % (func.__name__, time.time() - local_time))
        return output
    return wrapper


class MathTicTacToe:
    def __init__(self):
        self.board = np.zeros((3, 3), dtype=int)
        self.available_numbers = list(range(1, 10))
        self.score_cache = {}
        self.hit = 0

    def print_board(self):
        for row in self.board:
            print(" | ".join(str(num) if num != 0 else ' ' for num in row))
            print("---------")


    def check_win(self):

        # Create a mask to check non-zero elements (i.e., filled cells)
        non_zero_mask = self.board != 0

        # Check rows and columns for sum of 15 and all non-zero
        row_sums = np.sum(self.board, axis=1)
        col_sums = np.sum(self.board, axis=0)

        # Use the mask to ensure all elements in the row or column are non-zero
        valid_rows = np.all(non_zero_mask, axis=1)
        valid_cols = np.all(non_zero_mask, axis=0)

        # Check the rows and columns if any are valid and sum to 15
        if np.any((row_sums == 15) & valid_rows):
            return True
        if np.any((col_sums == 15) & valid_cols):
            return True

        # Check diagonals
        main_diag = self.board.diagonal()
        anti_diag = np.fliplr(self.board).diagonal()

        if np.all(main_diag) and np.sum(main_diag) == 15:
            return True
        if np.all(anti_diag) and np.sum(anti_diag) == 15:
            return True

        return False

    def get_available_moves(self):
        moves = []

        for i in range(3):
            for j in range(3):
                if self.board[i][j] == 0:  # if the spot is empty
                    for number in self.available_numbers:
                        moves.append((i, j, number))

        if self.is_empty:
            # remove 1,1,5 from available_numbers at the beginning of the game
            moves.remove((1, 1, 5))

        return moves

    def make_move(self, move):
        # move should be a tuple (row, column, number)
        x, y, num = move
        self.board[x][y] = num
        self.available_numbers.remove(num)

    def undo_move(self, move):
        # move should be a tuple (row, column, number)
        x, y, num = move
        self.board[x][y] = 0
        self.available_numbers.append(num)

    @property
    def is_full(self):
        return len(self.available_numbers) == 0

    @property
    def is_empty(self):
        return np.all(self.board == 0)


### Runner Class

The Runner class aims to manage game tasks such as ai_vs_ai, human_vs_ai, and eval_case.

Importantly, it initializes with the search_func variable, which references either the min-max or alpha-beta pruning function to find the best move.

In [39]:
class Runner:
    def __init__(self, search_func):

        self.err = 0
        self.search_func = search_func

    def ai_vs_ai(self):

        game = MathTicTacToe()

        player = 1  # always set player 1 as the first player

        i = 0
        while not game.is_full:
            time_start = time.time()

            print(f'number of available moves for player {player}:', len(game.get_available_moves()))

            ai_move(game, player, search_func=self.search_func)

            game.print_board()
            if game.check_win():
                print(f'player {player} Win. Game OVER')
                break

            # change player after each move
            player *= -1
            i += 1

            # time print
            time_diff = time.time() - time_start
            print('Time:', f'{time_diff:.2f}')

        if not game.check_win():
            print('Tie!')

    def human_vs_ai(self, human_first=False):

        game = MathTicTacToe()

        player = 1  # always set player 1 as the first player

        remainder = 1 if human_first else 0
        player2name = {1: 'human' if human_first else 'AI', -1: 'AI' if human_first else 'human'}

        i = 0
        while not game.is_full:
            time_start = time.time()

            print(f'number of available moves for player {player}:', len(game.get_available_moves()))

            if i % 2 == remainder:
                ai_move(game, player, search_func=self.search_func)
            else:
                human_move(game)

            game.print_board()
            if game.check_win():
                print(f'player {player2name[player]} Wins. Game OVER')
                break

            # change player after each move
            player *= -1
            i += 1

            # time print
            time_diff = time.time() - time_start
            print('time:', f'{time_diff:.2f}')

        if not game.check_win():
            print('Tie!')

    @print_run_time # prints the running time of the function, equivalent to eval_case = print_run_time(eval_case)
    def eval_case(self, moves, next_move, score):
        """
        :param moves:  a list of moves [[row, col, num], [row, col, num], ...
        :param next_move: best move for the next player [row, col, num]
        :param score: the score of the next move
        usage: check whether search_func can correctly find the next move and score, and update results in self.err
        """
        game = MathTicTacToe()

        for i in range(len(moves)):
            game.make_move(moves[i])

        player = (-1) ** len(moves)

        ai_score, ai_next_move = ai_move(game, player, search_func=self.search_func, verbose=True)

        if (ai_score - score) ** 2 > 0.1 or ai_next_move != next_move:
            self.err += 1
            print(f'AI move incorrect: ai_move {ai_next_move}, true_move {next_move}, ai_score {ai_score}, true_score {score}')
        else:
            print('AI move correct')


### Move functions

Here, we provide two move functions: human_move and ai_move.

In [40]:
def human_move(game):

    """This function handles human input for a move in Math Tic-Tac-Toe."""
    valid_move = False
    move = None

    while not valid_move:
        # Display current board state and available numbers
        print("Available numbers:", game.available_numbers,flush=True)

        time.sleep(0.2)

        # Get input from the user
        input_str = input("Enter your move (row col num) (Enter 'q' to quit the game): ")
        try:
            if input_str.lower() == 'q':
                import sys
                sys.exit(0)

            row, col, num = map(int, input_str.split())
            move = (row, col, num)
            if move not in game.get_available_moves():
                print("Invalid move. Please try again.", flush=True)
                continue
            else:
                valid_move = True

        except ValueError:
            print("Invalid input. Please use the format 'row col num' with integers.")
        except Exception as e:
            print(f"Unexpected error: {e}. Please enter your move correctly.")

    # Make the move
    game.make_move(move)
    print("You placed", num, "at position", (row, col), flush=True)

    return move

def ai_move(game, player, search_func, verbose=False):
    """
    :param game: instance of MathTicTacToe for game status
    :param player: player 1 or -1
    :param search_func: min-max or min-max alpha-beta pruning function
    :param verbose: whether print the board and move
    :return: best_value, move (for evaluation purpose)
    """

    if verbose:
        print('-' * 20)
        print('AI player as player ', player)
        print('current board')
        game.print_board()

    best_value = None

    if game.is_empty:
        # AI first move with random choice, since the first move is always winning. We can choose any move.
        move = random.choice(game.get_available_moves())
        game.make_move(move)
        print('AI first move with random choice: ', move)
    else:
        best_value, move = search_func(game, player)
        game.make_move(move)
        print('AI current move:', move, '; score:', best_value)

    return best_value, move


### Check the game board

In [41]:
# check the game Class

game = MathTicTacToe()

# make move
game.make_move((1,2,5))
print('-'*30)
game.print_board()
print(f"Check win status: {game.check_win()}")

# make move
game.make_move((1,1,6))
print('-'*30)
game.print_board()
print(f"Check win status: {game.check_win()}")

# make move
print('-'*30)
game.make_move((1,0,4))
game.print_board()

print(f"Check win status: {game.check_win()}")
print(f"Available moves: {game.get_available_moves()}")
print(f"Is the board full? {game.is_full}")
print(f"Is the board empty? {game.is_empty}")

------------------------------
  |   |  
---------
  |   | 5
---------
  |   |  
---------
Check win status: False
------------------------------
  |   |  
---------
  | 6 | 5
---------
  |   |  
---------
Check win status: False
------------------------------
  |   |  
---------
4 | 6 | 5
---------
  |   |  
---------
Check win status: True
Available moves: [(0, 0, 1), (0, 0, 2), (0, 0, 3), (0, 0, 7), (0, 0, 8), (0, 0, 9), (0, 1, 1), (0, 1, 2), (0, 1, 3), (0, 1, 7), (0, 1, 8), (0, 1, 9), (0, 2, 1), (0, 2, 2), (0, 2, 3), (0, 2, 7), (0, 2, 8), (0, 2, 9), (2, 0, 1), (2, 0, 2), (2, 0, 3), (2, 0, 7), (2, 0, 8), (2, 0, 9), (2, 1, 1), (2, 1, 2), (2, 1, 3), (2, 1, 7), (2, 1, 8), (2, 1, 9), (2, 2, 1), (2, 2, 2), (2, 2, 3), (2, 2, 7), (2, 2, 8), (2, 2, 9)]
Is the board full? False
Is the board empty? False


## Minimax algorithm

In this task, you will implement a minimax algorithm for the math Tic-Tac-Toe game.

If player1 wins, the terminal score will be 100. Otherwise, the terminal score will be -100. (i.e., The player1 is a maximizer, while player2 is a minimizer.). The terminal score will be 0 for a tie.

In this game, we consider the search depth as well. Intuitively, the score with a larger search depth will be discounted. We consider a decay factor for the discounted score. That is, $return\_score = score * decay\_factor^{depth-1}$. (This implementation is provided to simplify the tasks.)


### Task1: Implement minimax

- In this task, you will implement  `min_max_cached` functions. Please ensure you have addressed all of the [TODO] items  before submission.
- Unlike the traditional minimax algorithm, we use a cache to store the score of the board status. If the board status has been calculated before, we will directly return the score from the cache.
- We also consider the depth of the search. The score will be discounted by a decay factor. When you recursively call the function, the depth will be increased by 1.


**Task 1 [TODO] list :** \

**TODO 1.1**: Retrieve all possible moves and scores, and find the move with the best score in the maximizer's view.

**TODO 1.2**: Retrieve all possible moves and scores, and find the move with the best score in the minimizer's view.




In [42]:
def min_max_cached(game, player, depth=0):
    """
    :param game: an instance of MathTicTacToe
    :param player: whether maximizing player(1) or minimizing player(-1)
    :param depth: current search depth
    :return: best_value, best_move
    """

    # cache_key is a tuple of the board status and player
    cache_key = (tuple(game.board.flatten().tolist()), player)

    # if the board status has been calculated before, return the score from the cache
    if cache_key in game.score_cache:
        game.hit += 1
        return game.score_cache[cache_key]

    # boundary case check: we use decayed score for the leaf node
    if game.check_win():
        previous_player = -player
        score = 100 if previous_player == 1 else -100
        decay_factor = 0.9
        score *= decay_factor ** (depth - 1)

        game.score_cache[cache_key] = (score, None)
        return score, None

    if game.is_full:
        score = 0
        game.score_cache[cache_key] = (score, None)
        return score, None  # This is a draw

    best_value = float('-inf') * player  # inf for player -1, -inf for player 1
    best_move = None

    # scan all cases by min-max alpha-beta pruning
    if player == 1:
        # Maximizing player
        ######################## TODO 1.1 ############################
        # In this task, you search all possible moves and scores, and find the move with the best score in maximizer's view.
        # possible usage: game.get_available_moves(), game.make_move(), game.undo_move()
        # target variables to get: best_value, best_move
        # hint: you may use min_max_cached function recursively









        ######################## END of TODO 1.1 ############################

        game.score_cache[cache_key] = (best_value, best_move)# cache record
        return best_value, best_move
    else:
        # Minimizing player
        ######################## TODO 1.2 ############################
        # In this task, you search all possible moves and scores, and find the move with the best score in minimizer's view.
        # possible usage: game.get_available_moves(), game.make_move(), game.undo_move()
        # target variables: best_value, best_move
        # hint: you may use min_max_cached function recursively










        ######################## END of TODO 1.2 ############################

        game.score_cache[cache_key] = (best_value, best_move) # cache record
        return best_value, best_move



### Evaluation on Task 1

In [43]:
if __name__ == '__main__':

    # obtain the test cases
    with open('test_cases_tutorial.txt', 'r') as f:
        data = f.readlines()
        # print(data)
        entries = [line.strip() for line in data if line.strip()]

        # Convert each entry to a tuple using eval
        # each entry is in the form of (moves, next_move, score)
        list_of_tuples = [eval(entry) for entry in entries]

    runner = Runner(search_func=min_max_cached)

    n_test = len(list_of_tuples)
    for i in range(n_test):
        runner.eval_case(*list_of_tuples[i])

    print(f'Total error times: ',runner.err)
    print(f'Error rate: {runner.err / n_test * 100:.2f}%')

--------------------
AI player as player  -1
current board
  |   |  
---------
5 | 2 |  
---------
9 | 3 | 7
---------
AI current move: (0, 0, 1) ; score: -100.0
AI move correct
Current Function [eval_case] run time is 0.02 s
--------------------
AI player as player  -1
current board
9 |   | 4
---------
3 |   | 8
---------
2 |   |  
---------
AI current move: (1, 1, 6) ; score: 0
AI move correct
Current Function [eval_case] run time is 0.03 s
--------------------
AI player as player  -1
current board
  |   |  
---------
5 | 2 |  
---------
9 | 3 | 7
---------
AI current move: (0, 0, 1) ; score: -100.0
AI move correct
Current Function [eval_case] run time is 0.02 s
--------------------
AI player as player  1
current board
  |   |  
---------
7 |   | 1
---------
4 |   | 9
---------
AI current move: (0, 2, 5) ; score: 100.0
AI move correct
Current Function [eval_case] run time is 0.11 s
--------------------
AI player as player  1
current board
7 |   |  
---------
  |   | 6
---------
4 | 1

## Alpha-beta pruning

In this task, you will implement the alpha-beta pruning algorithm for the math Tic-Tac-Toe game.

The alpha-beta pruning algorithm is an optimization of the minimax algorithm. It reduces the number of nodes evaluated in the search tree by eliminating branches guaranteed to be worse than the best move found so far.

### Task2: Implement Alpha-beta Pruning

- In this task, you will implement  `min_max_alpha_beta_cached` functions. Please ensure you have addressed all of the [TODO] items  before submission.


- Unlike the traditional minimax algorithm, we use a cache to store the score of the board status.
- The cache recording is slightly different from minimax, since we can only record cache when alpha-beta pruning is not triggered. (The score is not accurate when alpha-beta pruning is triggered, since we do not search all the possible moves.)
- In alpha-beta pruning, we use two extra parameters, alpha and beta, to keep track of the best score the maximizer can guarantee at that level or above and the best score the minimizer can guarantee at that level or above, respectively.
- We also consider the depth in the search. The score will be discounted by a decay factor. When you recursively call the function, the depth will be increased by 1.


**Task 2 [TODO] list :** \

**TODO 2.1**: Retrieve all possible moves and scores, and find the move with the best score in the maximizer's view with alpha-beta pruning.

**TODO 2.2**: Retrieve all possible moves and scores, and find the move with the best score in the minimizer's view with alpha-beta pruning.


In [44]:
def min_max_alpha_beta_cached(game, player, alpha=float('-inf'), beta=float('inf'), depth=0):

    # cache_key is a tuple of the board status and player
    cache_key = (tuple(game.board.flatten().tolist()), player)

    if cache_key in game.score_cache:
        game.hit += 1
        return game.score_cache[cache_key]

    # boundary case check: we use decayed score for the leaf node
    if game.check_win():
        previous_player = -player
        score = 100 if previous_player == 1 else -100
        decay_factor = 0.9
        score *= decay_factor ** (depth - 1)

        game.score_cache[cache_key] = (score, None)
        return score, None

    if game.is_full:
        score = 0
        game.score_cache[cache_key] = (score, None)
        return score, None  # This is a draw

    best_value = float('-inf') * player  # inf for player -1, -inf for player 1
    best_move = None
    alpha_beta_break = False

    # scan all cases by min-max alpha-beta pruning
    if player == 1:

        # Maximizing player
        ######################## TODO 2.1 ############################
        # In this task, you search all possible moves and scores, and find the move with the best score in maximizer's view with alpha-beta pruning.
        # possible usage: game.get_available_moves(), game.make_move(), game.undo_move()
        # target variables to get: best_value, best_move, alpha_beta_break
        # hint: you may use min_max_alpha_beta_cached function recursively;
        # hint: you need to change alpha_beta_break flag when alpha-beta pruning is triggered.










        ######################## END of TODO 2.1 ############################

        if not alpha_beta_break:
            game.score_cache[cache_key] = (best_value, best_move)

        return best_value, best_move

    else:
        # Minimizing player

        ######################## TODO 2.2 ############################
       # In this task, you search all possible moves and scores, and find the move with the best score in minimizer's view with alpha-beta pruning.
       # possible usage: game.get_available_moves(), game.make_move(), game.undo_move()
        # target variables to get: best_value, best_move, alpha_beta_break
        # hint: you may use min_max_alpha_beta_cached function recursively; you need to change alpha_beta_break flag when alpha-beta pruning is triggered.












        ######################## END of TODO 2.2 ############################
        if not alpha_beta_break:
            game.score_cache[cache_key] = (best_value, best_move)
        return best_value, best_move

### Evaluation on Task 2

In [45]:
if __name__ == '__main__':

    with open('test_cases_tutorial.txt', 'r') as f:
        data = f.readlines()
        print(data)
        entries = [line.strip() for line in data if line.strip()]

        # Convert each entry to a tuple using eval
        list_of_tuples = [eval(entry) for entry in entries]

    runner = Runner(search_func=min_max_alpha_beta_cached)

    n_test = len(list_of_tuples)
    for i in range(n_test):
        runner.eval_case(*list_of_tuples[i])
    print(f'Total error times: ',runner.err)
    print(f'Error rate: {runner.err / n_test * 100:.2f}%')

['([(1, 0, 5), (2, 0, 9), (2, 1, 3), (1, 1, 2), (2, 2, 7)], (0, 0, 1), -100.0)\n', '([(2, 0, 2), (0, 2, 4), (0, 0, 9), (1, 0, 3), (1, 2, 8)], (1, 1, 6), 0)\n', '([(1, 0, 5), (2, 0, 9), (2, 1, 3), (1, 1, 2), (2, 2, 7)], (0, 0, 1), -100.0)\n', '\n', '([(1, 0, 7), (2, 2, 9), (1, 2, 1), (2, 0, 4)], (0, 2, 5), 100.0)\n', '([(2, 1, 1), (1, 2, 6), (2, 0, 4), (0, 0, 7)], (1, 0, 5), 65.61)\n', '([(1, 0, 5), (0, 1, 9), (1, 2, 6), (0, 2, 3)], (1, 1, 4), 100.0)\n', '\n', '([(0, 0, 5), (2, 1, 4), (0, 1, 8)], (0, 2, 2), -100.0)\n', '([(1, 2, 8), (0, 1, 3), (1, 0, 9)], (0, 0, 6), 0)\n', '([(0, 2, 5), (1, 0, 2), (2, 1, 4)], (0, 0, 8), 72.9)\n', '\n', '([(0, 2, 7), (1, 2, 1)], (0, 0, 4), 65.61)\n', '([(1, 2, 9), (0, 1, 8)], (2, 0, 7), 81.0)\n']
--------------------
AI player as player  -1
current board
  |   |  
---------
5 | 2 |  
---------
9 | 3 | 7
---------
AI current move: (0, 0, 1) ; score: -100.0
AI move correct
Current Function [eval_case] run time is 0.02 s
--------------------
AI player as pl

## Play with implemented AI

### Task 3 (Optional)
After you implement two tasks, you can play with the AI.

You can set the `human_first` parameter to `True` or `False` to decide whether the human or AI will make the first move.

In [46]:
if __name__ == '__main__':
    runner = Runner(search_func=min_max_alpha_beta_cached)
    runner.human_vs_ai(human_first=False)

number of available moves for player 1: 80
AI first move with random choice:  (2, 0, 4)
  |   |  
---------
  |   |  
---------
4 |   |  
---------
time: 0.00
number of available moves for player -1: 64
Available numbers: [1, 2, 3, 5, 6, 7, 8, 9]
Enter your move (row col num) (Enter 'q' to quit the game): 1 1 5
You placed 5 at position (1, 1)
  |   |  
---------
  | 5 |  
---------
4 |   |  
---------
time: 7.15
number of available moves for player 1: 49
AI current move: (0, 2, 6) ; score: 100.0
  |   | 6
---------
  | 5 |  
---------
4 |   |  
---------
player AI Wins. Game OVER
